<a href="https://colab.research.google.com/github/Vihelisa/dados.Python/blob/main/Converte_TXT_para_daily_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files 
uploaded = files.upload() 
keys = list(uploaded.keys())
real_file = uploaded[keys[0]]

#keys é o nome do arquivo

Saving datalog_A5.txt to datalog_A5.txt


In [10]:
def dividir(div):
 resultado = DadoLinhaInteira.split('M')
 return resultado[0], resultado[1]


def convert(hexT, hexU):
  separarT, separarU = [hexT.split('.'), hexU.split('.')]
  separar_frenteT = str(int(separarT[0], 16)) 
  separar_trasT = str(int(separarT[1], 16)) 
  separar_frenteU = str(int(separarU[0], 16)) 
  separar_trasU = str(int(separarU[1], 16)) 
  TEMP = separar_frenteT + '.' + separar_trasT
  UMI = separar_frenteU + '.' + separar_trasU 
  T_U = [TEMP, UMI]
  return T_U


def data_in_date(data):
  data_inteira = Dado_Data + '/2021'
  data_inteira = datetime.strptime(data_inteira, '%d/%m/%Y').date()
  return data_inteira

  
def hora_in_time (hora):
  hora_inteira = Dado_Hora + ':00'
  hora_inteira = datetime.strptime(hora_inteira, '%H:%M:%S').time()
  return hora_inteira


main

-Abrindo arquivo

In [11]:
#path = uploaded
#ref_arquivo = open(path, 'r')
#arquivi_convertido = ref_arquivo.read()

file_as_list = real_file.split()
converted_list = []
for byte_element in file_as_list:
  converted_list.append(byte_element.decode("utf-8"))


####
#TEMPORARIO
arquivo_list = converted_list
#####

import pandas as pd 
from datetime import datetime
from datetime import date
df = pd.DataFrame()

for DadoLinhaInteira in arquivo_list:
  if 'MID_' in DadoLinhaInteira:
  #if 0:
    print('ReStart')
  else:
    periodo, dados = dividir(DadoLinhaInteira)
    Dado_Data, Dado_Hora = periodo.split('_')
    Dado_D = data_in_date(Dado_Data)
    Dado_H = hora_in_time(Dado_Hora)
    Dado_DH = [Dado_D, Dado_H]
    Dado_hexT, Dado_hexU = dados.split('P')
    Dado_TU = convert(Dado_hexT, Dado_hexU)
    lista_dados = [Dado_DH[0], Dado_DH[1], Dado_TU[0], Dado_TU[1]]
    dados_dict =  { 'Data': lista_dados[0], 'Hora':lista_dados[1], 'Temperatura':lista_dados[2], 'Umidade':lista_dados[3]}
    df = df.append(dados_dict, ignore_index=True)
    #print(dados_dict)


df['Time'] = df['Data'].map(str) + ' ' + df['Hora'].map(str)
df['delta'] = df['Time']
df.set_index('Time', inplace = True)
df['etto'] = pd.to_datetime(df['delta'])
#convert data from string to float
df['tempf'] = df['Temperatura'].apply(pd.to_numeric)
df['humi'] = df['Umidade'].apply(pd.to_numeric)


ReStart
ReStart
ReStart
ReStart


In [13]:
#separate by hour
df_ok = df.copy()
#print(df_ok.shape)
temperature_only = df_ok.resample('H',on='etto').tempf.mean()
humidity_only = df_ok.resample('H',on='etto').humi.mean()

#join it on a dataframe
temperature_frame = temperature_only.to_frame()
humidity_frame = humidity_only.to_frame()
df_data_by_hour = temperature_frame.join(humidity_frame)


import numpy as np

#add molhamento
df_data_by_hour['molhamento'] = '0'

#add t_dpm
df_data_by_hour['t_dpm'] = '0'

for index, row in df_data_by_hour.iterrows():
  if float(row[1]) > 90:
    #print(row[1])
    #df_data_by_hour['molhamento'][row] = 1
    df_data_by_hour.loc[index, 'molhamento'] = 1
    df_data_by_hour.loc[index, 't_dpm'] = float(row[0])
  else:
    df_data_by_hour.loc[index, 't_dpm'] = np.NaN


#Finish the hour dataframe
df_complete_hour = df_data_by_hour.copy()
df_complete_hour.columns = ['Temperature','Humidity','Molhamento', 'T_DPM']
#df_complete_hour.head() 

df_buffer_by_day = df_complete_hour.copy()

#convert to numeric to avoid errors
df_buffer_by_day['Molhamento_integer'] = df_buffer_by_day['Molhamento'].apply(pd.to_numeric)
df_buffer_by_day['T_DPM_integer'] = df_buffer_by_day['T_DPM'].apply(pd.to_numeric)

#separates by day
temperature_mean_by_day = df_buffer_by_day.resample('D').Temperature.mean()
humidity_mean_by_day = df_buffer_by_day.resample('D').Humidity.mean()
molhamento_sum_by_day = df_buffer_by_day.resample('D').Molhamento_integer.sum()
T_DPM_MEAN_by_day = df_buffer_by_day.resample('D').T_DPM_integer.mean()

#concat all series
isolated_series = {'Temperature': temperature_mean_by_day, 'Humidity':humidity_mean_by_day, 'Molhamento':molhamento_sum_by_day, 'T_DPM':T_DPM_MEAN_by_day}
df_final_result = pd.DataFrame(isolated_series)


In [14]:
#df_final_result.to_csv("data_by_day_1.csv")
output_file_name = keys[0][:-4] 
df_final_result.to_csv(output_file_name+ '.csv', index=False)
df_final_result.head()

,Temperature,Humidity,Molhamento,T_DPM
etto,,,,
2021-01-05,23.259394,85.128788,5,20.996000
2021-01-06,27.558562,67.513056,4,20.358750
2021-01-07,26.408492,59.500625,0,NaN
2021-01-08,27.213204,47.773373,0,NaN
2021-01-09,26.290694,65.449792,2,21.773333
